In [0]:
%scala
val nbSuffix="_demo3"

nbSuffix: String = _demo3

In [0]:
%sh
cat /dbfs/FileStore/conf/spark_v1_0_offline.conf

[kensu]
;Kensu Ingestion endpoint
kensu_ingestion_url=https://community-api.kensuapp.com
;token for Kensu Ingestion API
kensu_ingestion_token=
;token for Kensu Expectation API (external app personal access token)
kensu_api_token=

; if report_to_file=True then no credentials needed
report_to_file=True
offline_file_name=demo_notebook3.jsonl

project_name=Databricks demo v2
environment=Databricks dev
logical_data_source_naming_strategy=LastFolderAndFile


compute_stats=True
compute_input_stats=True
compute_output_stats=True

; these are set automatically for Databricks notebooks:
; user_name=databricks user
; code_location,process_name=Notebook name

In [0]:
%scala
io.kensu.sparkcollector.environments.DatabricksCollector.track(spark, "/dbfs/FileStore/conf/spark_v1_0_offline.conf")

Collector's log file not enabled
Spark version: 3.2.1
Collector version: { assembly_assemblyJarName: kensu-spark-collector-1.0.13-SNAPSHOT_spark-3.2.1.jar, version: 1.0.13-SNAPSHOT, scalaVersion: 2.12.17, gitHeadCommit: Some(927af7b39adda26f3686046635121d52ae782bdd), builtAtString: 2022-10-26 11:15:02.659+0300, builtAtMillis: 1666772102659 }
Creating a new Kensu Spark-Collector actor system
Done creating a new Kensu Spark-Collector actor system

In [0]:
%scala
import org.apache.spark.sql.functions.expr;

def loadStock(year:String, month:String, stock:String):org.apache.spark.sql.DataFrame = {
  spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(s"/FileStore/Financial_Data_Report/$year/$month/$stock.csv")
    .withColumnRenamed("Adj Close", "AdjClose")
    .withColumn("Intraday_Delta", expr("AdjClose - Open"))
}

import org.apache.spark.sql.functions.expr
loadStock: (year: String, month: String, stock: String)org.apache.spark.sql.DataFrame

In [0]:
%scala
val year = "2021"
val month = "dec"

val apple_stock_df = loadStock(year,month, "Apple")
val apptech_stock_df = loadStock(year,month, "AppTech")
val buzzfeed_stock_df = loadStock(year,month, "Buzzfeed")
val eur_usd_df = loadStock(year,month, "Apple")
val imetal_df = loadStock(year,month, "AppTech")
val microsoft_stock_df = loadStock(year,month, "Buzzfeed")

//display(microsoft_stock_df)
val dfSeq = Seq(apple_stock_df, apptech_stock_df, buzzfeed_stock_df,eur_usd_df,imetal_df,microsoft_stock_df)
val monthly_assets_df = dfSeq.reduce(_ union _)

monthly_assets_df.registerTempTable("tmp_monthly_assets")
//monthly_assets_df.show()

command-266451873421926:15: warning: method registerTempTable in class Dataset is deprecated (since 2.0.0): Use createOrReplaceTempView(viewName) instead.
monthly_assets_df.registerTempTable("tmp_monthly_assets")
 ^
year: String = 2021
month: String = dec
apple_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
apptech_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
buzzfeed_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
eur_usd_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
imetal_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
microsoft_stock_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]
dfSeq: Seq[org.apache.spark.sql.DataFrame] = List([Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields], [Date: timestamp, Symbol: string ... 7 more fields])
monthly_assets_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Symbol: string ... 7 more fields]

In [0]:
%sql
DROP TABLE IF EXISTS financial_data_monthly_assets_demo1


In [0]:
%sql
CREATE OR REPLACE TABLE financial_data_monthly_assets_demo1 USING DELTA AS SELECT * FROM tmp_monthly_assets


num_affected_rows,num_inserted_rows


In [0]:
%scala
// Csv read, Databricks Delta write
spark.sql("select * from tmp_monthly_assets").write.mode("overwrite").format("delta").saveAsTable("financial_data_monthly_assets_vidma" + nbSuffix)

In [0]:
%scala
// Databricks Delta read and Delta write
spark.sql("select * from financial_data_monthly_assets_vidma" + nbSuffix).write.mode("overwrite").format("delta").saveAsTable("delta_read_and_delta_write_vidma_" + nbSuffix)

In [0]:
%sh
ls -lhrt *.jsonl

-rw-r--r-- 1 root root 35K Oct 26 08:26 demo_notebook3.jsonl

In [0]:
%sh

cat demo_notebook3.jsonl

{"action":"add_entity","entity":"BATCH_ENTITY_REPORT","generatedEntityGUID":"6fa60544-3cd0-4c3f-8581-91d771af70e0","schemaVersion":"0.1","jsonPayload":{"projects":[{"timestamp":1666766331735,"entity":{"pk":{"name":"Databricks demo v2"}}}],"processes":[{"timestamp":1666766331789,"entity":{"pk":{"qualifiedName":"/Users/vidma@kensu.io/demo/demo_load_financial_data_copy3_offline"}}}],"processRuns":[{"timestamp":1666766331904,"entity":{"pk":{"processRef":{"byGUID":"k-0da80907c06313c43f9767da29aea97a8f13f350a5b885ae070c5aeb1e5c7591"},"qualifiedName":"app-20221026061720-0000"},"launchedByUserRef":{"byGUID":"k-66cd9823fd5eb060f0b66f5f2d98ef6b7e1bc4af24b46e9233a2258dabde6314"},"environment":"Databricks dev","projectsRefs":[{"byGUID":"k-67d8ae8700765f61e92b33ceacf5a8dc5583acc77395eebb02c9391f599c058c"}]}}],"processLineages":[{"timestamp":1666766331907,"entity":{"name":"Generate `schema::delta: default.financial_data_monthly_assets_vidma_demo3` from `schema::dec/AppTech.csv`, `schema::dec/Apple.csv`, `schema::dec/Buzzfeed.csv`","operationLogic":"APPEND","pk":{"processRef":{"byGUID":"k-0da80907c06313c43f9767da29aea97a8f13f350a5b885ae070c5aeb1e5c7591"},"dataFlow":[{"fromSchemaRef":{"byGUID":"k-4fd568346c83f526defc6d300eb65cf49dccd961442256fb236e62bf043874e5"},"toSchemaRef":{"byGUID":"k-22000626e7f789e994548a5d9cebbf0a5166894408641ad06a116e2cd37aa9fb"},"columnDataDependencies":{"Volume":["Volume","Volume"],"Intraday_Delta":["Adj Close","Open","Adj Close","Open"],"High":["High","High"],"Close":["Close","Close"],"Symbol":["Symbol","Symbol"],"Open":["Open","Open"],"Date":["Date","Date"],"AdjClose":["Adj Close","Adj Close"],"Low":["Low","Low"]}},{"fromSchemaRef":{"byGUID":"k-4f681cedb39d92dc42e4fda0b3fc56ee5855b8123ea02aa51ac55c00f10065a0"},"toSchemaRef":{"byGUID":"k-22000626e7f789e994548a5d9cebbf0a5166894408641ad06a116e2cd37aa9fb"},"columnDataDependencies":{"Volume":["Volume","Volume"],"Intraday_Delta":["Adj Close","Open","Adj Close","Open"],"High":["High","High"],"Close":["Close","Close"],"Symbol":["Symbol","Symbol"],"Open":["Open","Open"],"Date":["Date","Date"],"AdjClose":["Adj Close","Adj Close"],"Low":["Low","Low"]}},{"fromSchemaRef":{"byGUID":"k-c388f043c5680795c2a881ffb5f36a754edcdcca4905e7c0fedec4d0eb055fcc"},"toSchemaRef":{"byGUID":"k-22000626e7f789e994548a5d9cebbf0a5166894408641ad06a116e2cd37aa9fb"},"columnDataDependencies":{"Volume":["Volume","Volume"],"Intraday_Delta":["Adj Close","Open","Adj Close","Open"],"High":["High","High"],"Close":["Close","Close"],"Symbol":["Symbol","Symbol"],"Open":["Open","Open"],"Date":["Date","Date"],"AdjClose":["Adj Close","Adj Close"],"Low":["Low","Low"]}}]}}}],"lineageRuns":[{"timestamp":1666766331908,"entity":{"pk":{"processRunRef":{"byGUID":"k-6b12945e54f69f3d628fc1309ee13632666350d958cfa65c90815d74e811bcbb"},"lineageRef":{"byGUID":"k-16a35819bb355da6af971f7b866c8984a3875babeef48c93e7cdf7402388528b"},"timestamp":1666766331908}}}],"schemas":[{"timestamp":1666766331903,"entity":{"name":"schema::delta: default.financial_data_monthly_assets_vidma_demo3","pk":{"dataSourceRef":{"byGUID":"k-34c6c71df9e169f9d8361674309bd4d39777827d475fe5d66074534dd9a13b3b"},"fields":[{"name":"Date","fieldType":"timestamp","nullable":true},{"name":"Symbol","fieldType":"string","nullable":true},{"name":"Open","fieldType":"double","nullable":true},{"name":"High","fieldType":"double","nullable":true},{"name":"Low","fieldType":"double","nullable":true},{"name":"Close","fieldType":"double","nullable":true},{"name":"AdjClose","fieldType":"double","nullable":true},{"name":"Volume","fieldType":"integer","nullable":true},{"name":"Intraday_Delta","fieldType":"double","nullable":true}]}}},{"timestamp":1666766331901,"entity":{"name":"schema::dec/Buzzfeed.csv","pk":{"dataSourceRef":{"byGUID":"k-cfcee0eb244acc05c969e01da81e23ea71f1d82d140e6c99f69f91650bdfd8d9"},"fields":[{"name":"Date","fieldType":"timestamp","nullable":true},{"name":"High","fieldType":"double","nullable":true},{"name":"Close","fieldType":"double","nullable":true},{"name"